In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi,cos,sin
from scipy.linalg import logm
from scipy.optimize import least_squares
import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import showinfo

In [2]:
#将从示教器读取的数据转化为旋转矩阵和平移矩阵
def three_to_R(M):
    T=np.array(M[:3]).reshape(-1,1)
    rota=M[3]*pi/180
    rotb=M[4]*pi/180
    rotc=M[5]*pi/180
    Rx=np.array([[1,0,0],[0,cos(rota),-sin(rota)],[0,sin(rota),cos(rota)]])
    Ry=np.array([[cos(rotb),0,sin(rotb)],[0,1,0],[-sin(rotb),0,cos(rotb)]])
    Rz=np.array([[cos(rotc),-sin(rotc),0],[sin(rotc),cos(rotc),0],[0,0,1]])
    R=np.dot(Rz,np.dot(Ry,Rx))
    h_stack=np.hstack((R,T))
    one=np.array([0,0,0,1])
    TR=np.vstack((h_stack,one))
    return TR

In [3]:
#将四元数转化旋转矩阵和平移矩阵
def four_to_R(M):
    t1,t2,t3,q0,q1,q2,q3,=M
    x11=q0**2+q1**2-q2**2-q3**2
    x12=2*(q1*q2-q0*q3)
    x13=2*(q1*q3+q0*q2)
    x14=t1
    x21=2*(q1*q2+q0*q3)
    x22=q0**2-q1**2+q2**2-q3**2
    x23=2*(q2*q3-q0*q1)
    x24=t2
    x31=2*(q1*q3-q0*q2)
    x32=2*(q2*q3+q0*q1)
    x33=q0**2-q1**2-q2**2+q3**2
    x34=t3
    TR=np.array([[x11,x12,x13,x14],[x21,x22,x23,x24],[x31,x32,x33,x34],[0,0,0,1]])
    return TR

In [4]:
#将极坐标转换为直角坐标
def axis_transformtion(x):
    result=[]
    for i in [0,2,4,6]:
        x0=x[i]*cos(x[i+1]/180*pi)
        y0=x[i]*sin(x[i+1]/180*pi)
        result.append(x0)
        result.append(y0)
    return result

In [5]:
def calculate_fixed_point(fix_point,R,position):
    k1=(fix_point[3]-fix_point[1])/(fix_point[2]-fix_point[0])#k1=(y2-y1)/(x2-x1)
    k2=(fix_point[7]-fix_point[5])/(fix_point[6]-fix_point[4])#k2=(y4-y3)/(x4-x3)
    k11=-1/k1#垂直平分线1的斜率
    k22=-1/k2#垂直平分线2的斜率
    x01=(fix_point[0]+fix_point[2])/2#x01=(x1+x2)/2——中点1的横坐标
    y01=(fix_point[1]+fix_point[3])/2#y01=(y1+y2)/2——中点1的纵坐标
    x02=(fix_point[4]+fix_point[6])/2#x02=(x3+x4)/2——中点2的横坐标
    y02=(fix_point[5]+fix_point[7])/2#y02=(y3+y4)/2——中点2的纵坐标
    x=((k11*x01-y01)-(k22*x02-y02))/(k11-k22)
    y=k11*(x-x01)+y01#(x,y)为圆心O'的坐标
    r2=(x-fix_point[0])**2+(y-fix_point[1])**2
    #得到圆心的x,y坐标后还需要求解z坐标
    distance=(R**2-r2)**0.5#z方向的垂直距离
    if position==1:
        z=-distance
    else:
        z=distance
    circle_point=np.array([x,y,z,1]).reshape(-1,1)
    return circle_point

In [6]:
def calculate_P0(TB,X,P):
    P0=np.dot(TB,np.dot(X,P))
    return P0

In [7]:
#用于储存机械臂和激光雷达的测量数据
A=[]
B=[]
def func(x):
        q0,q1,q2,q3,t1,t2,t3=x
        X=four_to_R(x)
        A1=np.dot(A[0],np.dot(X,B[0]))-np.dot(A[1],np.dot(X,B[1]))
        A2=np.dot(A[1],np.dot(X,B[1]))-np.dot(A[2],np.dot(X,B[2]))
        A3=np.dot(A[2],np.dot(X,B[2]))-np.dot(A[3],np.dot(X,B[3]))
        A4=np.dot(A[3],np.dot(X,B[3]))-np.dot(A[4],np.dot(X,B[4]))
        f=[A1[0,0],A1[1,0],A1[2,0],A2[0,0],A2[1,0],A2[2,0],A3[0,0],A3[1,0],A3[2,0],A4[0,0],A4[1,0],A4[2,0]]
        return f

In [8]:
def first_measure():
    measure_window = tk.Toplevel(root)
    #输入位姿1的数据
    Pose1=tk.StringVar()
    fix_point1=tk.StringVar()

    robot_label1=ttk.Label(measure_window,text="请输入机械臂位姿1的数据:")
    robot_label1.pack(fill="x",expand=True)
    robot_entry1=ttk.Entry(measure_window,textvariable=Pose1)
    robot_entry1.pack(fill="x",expand=True)
    robot_entry1.focus()
    
    #输入位姿1下激光雷达的数据
    radar_label1=ttk.Label(measure_window,text="请输入位姿1下激光雷达的数据:")
    radar_label1.pack(fill="x",expand=True)
    radar_entry1=ttk.Entry(measure_window,textvariable=fix_point1)
    radar_entry1.pack(fill="x",expand=True)
    radar_entry1.focus()
    
    def add_data():
        Pose=str(Pose1.get())
        Pose=list(map(float,Pose.split(",")))
        TB1=three_to_R(Pose)
        fix_point=str(fix_point1.get())
        fix_point=list(map(float,fix_point.split(",")))
        fix_point=axis_transformtion(fix_point)
        #计算圆心坐标
        R=246
        P1=calculate_fixed_point(fix_point,R,position=1)
        A.append(TB1)
        B.append(P1)
    #finish button
    first_measure=ttk.Button(measure_window,text="finish",command=add_data)
    first_measure.pack(fill='x', expand=True, pady=12)

In [9]:
def second_measure():
    measure_window = tk.Toplevel(root)
    Pose2=tk.StringVar()
    fix_point2=tk.StringVar()

    #输入位姿2的数据
    robot_label2=ttk.Label(measure_window,text="请输入机械臂位姿2的数据:")
    robot_label2.pack(fill="x",expand=True)
    robot_entry2=ttk.Entry(measure_window,textvariable=Pose2)
    robot_entry2.pack(fill="x",expand=True)
    robot_entry2.focus()
    

    #输入位姿2下激光雷达的数据
    radar_label2=ttk.Label(measure_window,text="请输入位姿2下激光雷达的数据:")
    radar_label2.pack(fill="x",expand=True)
    radar_entry2=ttk.Entry(measure_window,textvariable=fix_point2)
    radar_entry2.pack(fill="x",expand=True)
    radar_entry2.focus()
    
    def add_data():
        Pose=str(Pose2.get())
        Pose=list(map(float,Pose.split(",")))
        TB2=three_to_R(Pose)
        fix_point=str(fix_point2.get())
        fix_point=list(map(float,fix_point.split(",")))
        fix_point=axis_transformtion(fix_point)
        #计算圆心坐标
        R=246
        P2=calculate_fixed_point(fix_point,R,position=1)
        A.append(TB2)
        B.append(P2)
    #finish button
    finish_button=ttk.Button(measure_window,text="finish",command=add_data)
    finish_button.pack(fill='x', expand=True, pady=10)

In [10]:
def third_measure():
    measure_window = tk.Toplevel(root)
    Pose3=tk.StringVar()
    fix_point3=tk.StringVar()

    #输入位姿3的数据
    robot_label3=ttk.Label(measure_window,text="请输入机械臂位姿3的数据:")
    robot_label3.pack(fill="x",expand=True)
    robot_entry3=ttk.Entry(measure_window,textvariable=Pose3)
    robot_entry3.pack(fill="x",expand=True)
    robot_entry3.focus()
    
    #输入位姿3下激光雷达的数据
    radar_label3=ttk.Label(measure_window,text="请输入位姿3下激光雷达的数据:")
    radar_label3.pack(fill="x",expand=True)
    radar_entry3=ttk.Entry(measure_window,textvariable=fix_point3)
    radar_entry3.pack(fill="x",expand=True)
    radar_entry3.focus()

    def add_data():
        Pose=str(Pose3.get())
        Pose=list(map(float,Pose.split(",")))
        TB3=three_to_R(Pose)
        fix_point=str(fix_point3.get())
        fix_point=list(map(float,fix_point.split(",")))
        fix_point=axis_transformtion(fix_point)
        #计算圆心坐标
        R=246
        P3=calculate_fixed_point(fix_point,R,position=1)
        A.append(TB3)
        B.append(P3)
    #finish button
    finish_button=ttk.Button(measure_window,text="finish",command=add_data)
    finish_button.pack(fill='x', expand=True, pady=10)

In [11]:
def fourth_measure():
    measure_window = tk.Toplevel(root)
    Pose4=tk.StringVar()
    fix_point4=tk.StringVar()

    #输入位姿4的数据
    robot_label4=ttk.Label(measure_window,text="请输入机械臂位姿4的数据:")
    robot_label4.pack(fill="x",expand=True)
    robot_entry4=ttk.Entry(measure_window,textvariable=Pose4)
    robot_entry4.pack(fill="x",expand=True)
    robot_entry4.focus()

    #输入位姿4下激光雷达的数据
    radar_label=ttk.Label(measure_window,text="请输入位姿4下激光雷达的数据:")
    radar_label.pack(fill="x",expand=True)
    radar_entry=ttk.Entry(measure_window,textvariable=fix_point4)
    radar_entry.pack(fill="x",expand=True)
    radar_entry.focus()

    def add_data():
        Pose=str(Pose4.get())
        Pose=list(map(float,Pose.split(",")))
        TB4=three_to_R(Pose)
        fix_point=str(fix_point4.get())
        fix_point=list(map(float,fix_point.split(",")))
        fix_point=axis_transformtion(fix_point)
        #计算圆心坐标
        R=246
        P4=calculate_fixed_point(fix_point,R,position=1)
        A.append(TB4)
        B.append(P4)
    #finish button
    finish_button4=ttk.Button(measure_window,text="finish",command=add_data)
    finish_button4.pack(fill='x',expand=True, pady=10)

In [12]:
def fifth_measure():
    measure_window = tk.Toplevel(root)
    Pose5=tk.StringVar()
    fix_point5=tk.StringVar()

    #输入位姿5的数据
    robot_label5=ttk.Label(measure_window,text="请输入机械臂位姿5的数据:")
    robot_label5.pack(fill="x",expand=True)
    robot_entry5=ttk.Entry(measure_window,textvariable=Pose5)
    robot_entry5.pack(fill="x",expand=True)
    robot_entry5.focus()
    
    #输入位姿5下激光雷达的数据
    radar_label=ttk.Label(measure_window,text="请输入位姿5下激光雷达的数据:")
    radar_label.pack(fill="x",expand=True)
    radar_entry=ttk.Entry(measure_window,textvariable=fix_point5)
    radar_entry.pack(fill="x",expand=True)
    radar_entry.focus()

    def add_data():
        Pose=str(Pose5.get())
        Pose=list(map(float,Pose.split(",")))
        TB5=three_to_R(Pose)
        fix_point=str(fix_point5.get())
        fix_point=list(map(float,fix_point.split(",")))
        fix_point=axis_transformtion(fix_point)
        #计算圆心坐标
        R=246
        P5=calculate_fixed_point(fix_point,R,position=1)
        A.append(TB5)
        B.append(P5)
    #finish button
    finish_button=ttk.Button(measure_window,text="finish",command=add_data)
    finish_button.pack(fill='x', expand=True, pady=10)

In [13]:
def calculate_biaoding_matrix():
    calculate_window = tk.Toplevel(root)
    initial_guess=[0.5,0.5,0.5,0.5,0.01,0.01,0.01]
    result=least_squares(func,initial_guess)
    global X
    X=four_to_R(result.x)
    showinfo(title="Information",message=f"计算得到的标定矩阵为:{X}")

In [14]:
def error_analysis():
    error=[]
    circle_point=[]
    for TB,P in zip(A,B):
        P0=calculate_P0(TB,X,P)
        circle_point.append(P0)
    for i in range(len(circle_point)):
        error.append(circle_point[i]-circle_point[0])
    showinfo(title="Information",message=f"标定误差为:{error}")

In [15]:
root=tk.Tk()
root.title("Controller")

#获取数据
control_label=ttk.Label(root,text="获取数据")
control_label.pack()

#第一次测量
first_measure=ttk.Button(root,text="输入位姿1的测量数据",command=first_measure)
first_measure.pack(fill='x', expand=True, pady=10)

#第二次测量
second_measure=ttk.Button(root,text="输入位姿2的测量数据",command=second_measure)
second_measure.pack(fill='x', expand=True, pady=10)

#第三次测量
third_measure=ttk.Button(root,text="输入位姿3的测量数据",command=third_measure)
third_measure.pack(fill='x', expand=True, pady=10)

#第四次测量
fourth_measure=ttk.Button(root,text="输入位姿4的测量数据",command=fourth_measure)
fourth_measure.pack(fill='x', expand=True, pady=10)

#第五次测量
fifth_measure=ttk.Button(root,text="输入位姿5的测量数据",command=fifth_measure)
fifth_measure.pack(fill='x', expand=True, pady=10)

#计算标定矩阵
biaoding_matrix_button=ttk.Button(root,text="获取标定矩阵",command=calculate_biaoding_matrix)
biaoding_matrix_button.pack(fill='x', expand=True, pady=10)

#误差分析
error_analysis_button=ttk.Button(root,text="误差分析",command=error_analysis)
error_analysis_button.pack(fill='x', expand=True, pady=10)
root.mainloop()